# Lab 05 - Exploratory Data Analysis and Statistical Modelling

We have spent a bit of time discussion data handling, manipulation, and collection. We are going to get a little more into analyses of data now.

When we first receive a dataset, the best thing to do is *not* to run out and find the biggest, best statistical machine to go and analyze it. We first really need to dig into those data and understand what's happening with it. Where are the data from? What kind of processes went into producing it? What got censored from it? Was the censoring intentional or nonintentional? What kind of biases are embedded in the data?

These are a lot of questions, some of which we can answer partially from the data itself, some of which we have to try to answer by going to other places. What we're doing to do today is focus on what we can extract from the data itself.

### Data: More Tweets, More Votes

In this lab, we'll work with the [More Tweets, More Votes](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0079449) (MTMV) data (the original paper is [here](http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0079449), if you're curious). The basic idea behind the MTMV paper is that if a politician gets more tweets then they are more likely to get a higher vote share. The paper's first author is [Joseph DiGrazia](http://josephdigrazia.com/), a sociologist.

There are many different variables in the MTMV dataset. The most important variables are the *dependent variable* (that is the variable which we are trying to explain). In this case, this is vote share (`vote_share`), which uses Republicans as a reference group. The variable takes on values between 0 and 1, and is the percentage of votes that Republicans got in that election. 

The variables which may explain variation in the dependent variable are called the *independent variables*. In the MTMV dataset there are many different variables which may explain vote share, including:

- Incumbency (measured by 1 or 0, 1 if there was a Republican incumbent, 0 if not)
- Ideological leaning
- Age
- Education level
- Gender
- Race
- Media markets

Most critically, however, we want to measure whether Twitter mention share (`mshare`) has an effect on vote share. Again, the variable will use Republicans as a reference group. We will also pay special attention to incumbency (`rep_inc`) because this is often the best predictor of who wins an election.

First we'll load the the MTMV data. The argument `index_col` defines one of the existing columns in the file as an index column.

In [ ]:
import pandas as pd
import numpy as np
df_mtmv = pd.read_csv("data/mtmv_data_10_12.csv", index_col = 0)

## Exploratory Data Analysis (EDA)

Exploratory Data Analysis (or EDA) is the process of exploring data before getting to any kind of modeling stage. EDA can tell us a lot about the data before we even get to any type of formal modeling or testing any kind of hypotheses about the data.

If you remember what we did in lab 2, we began with summarizing some basic statistics about the variables we're interested in. We can use `describe` to describe the basic shape of the `mshare` and `vote_share` variables.

In [ ]:
df_mtmv['mshare'].describe()

In [ ]:
df_mtmv['vote_share'].describe()

The average vote share for Republicans is 50% and the same goes for the Twitter mention share. There's a bit of variation, though, and it looks like 25% and 75% quintiles can be pretty far from that mean.

Similarly, we can generate some information about how many elections involved Republican incumbents. Since `rep_inc` only takes the values 1 and 0, we should treat it like a categorical variable. Recall, we can use `value_counts` to generate some information about that.

In [ ]:
df_mtmv['rep_inc'].value_counts()

## Visualization

Visualization is a body of methods to present data visually. It's a main component of EDA. There are three main reasons to visualize: 1) explore data, 2) confirm a model, and 3) present results. Visualization can highlight some important variations in a dataset which may be obscured by only focusing on summary statistics or by using statistical models.

For instance, look below at tthe example of Anscombe quartet. The data are dispersed in various configurations. But the mean and linear trend line are the same in all.

![](img/anscombe.png)

To begin, we'll import all the necessary modules for visualization. We use the <code>matplotlib.pyplot</code> module to visualize. We will also import the `seaborn` module because it provides for some really good ways to look at our data which aren't included as part of `matplotlib.pyplot`. 

`plt.style.use('seaborn')` sets the default style to that used in seaborn. It makes it look cleaner overall.

Lastly, we also will use <code>%matplotlib inline</code> to indicate that we want graphs to appear in our notebook.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
%matplotlib inline

### Univariate visualization - Histograms

Univariate visualisations allow us to understand single variables. They let us get beyond looking only at the mean, median, and standard deviations of these variables. It should be one of the first parts of exploring a dataset.

The first method we can look at is the histogram. This plots how many values of a variable appear in separate equal-sized bins. So for instance, in a variable that represent percentage like `mshare`, values vary from 0 to 100. So we'll typically see bins of size 10. We can make sure that we get 10 bins by passing in a keyword argument.

In [ ]:
df_mtmv['mshare'].hist(bins = 10)

In [ ]:
df_mtmv['vote_share'].hist(bins = 10)

We can also compare different variables in a histogram side-by-side by subsetting the data frame.

In [ ]:
df_mtmv[['mshare', 'vote_share']].hist()  

Another way of doing this is to overlay the different variables on top of each other in the histogram. We pass the argument <code>alpha</code> and set it to a 0.5 (0 is completely transparent, 1 is completely opaque) so we can see the overlap.

In [ ]:
df_mtmv[['mshare', 'vote_share']].plot.hist(alpha = 0.75, bins = 20)

### Bivariate visualization

Now that we have a good idea about how each of the variables are distributed, we can start trying to relate two or more variables to each other. Say we have two variables. Which is the best kind of visualization we can use to show the relationship between one variable and another? It depends on whether a variable is *categorical* or *continuous*. A categorical is what it sounds like -- values for that variable are categories and they cannot be ordered logically. A continuous variable is a number. It can take number of real values. Examples are 1, 0, 0.3333, 3.141, etc.

Looking at only two variables, here's a quick guide on how to visualize the comparison:

1. Categorical-categorical - use a table and/or a heatmap
2. Categorical-continuous - swarmplots, violin plots, and box plots.
3. Continuous-continuous - scatter plot

I will describe each of these below.

### Crosstabs + heatmaps

A *heatmap* is effectively a table or matrix in which extreme values are represented by more intense colors. It resembles a infrared camera which detects heat on surfaces or bodies.

Oddly enough, the MTMV dataset only has one categorical variable (`rep_inc`). But we can create a new categorical variable from a continuous variable. Let's split up `med_hhinc`, or median household income, into four categories based on quartiles. So we'll have the bottom 25% of income, second bottom 25%... all the way up to top 25%. We've already used `describe` to generate those values. Let's do so here.

In [ ]:
df_mtmv['med_hhinc'].describe()

We can set the output of `describe` to a pandas Series.

In [ ]:
mtmv_describe = df_mtmv['med_hhinc'].describe()

We can assign the values to a new column, `med_hhinc_quartile`. Don't mind all the warnings below for now.

In [ ]:
df_mtmv['med_hhinc_quartile'] = 0

In [ ]:
df_mtmv['med_hhinc_quartile'][df_mtmv['med_hhinc'] <= mtmv_describe['25%']] = '1 - Lower 25%'

df_mtmv['med_hhinc_quartile'].loc[(df_mtmv['med_hhinc'] >  mtmv_describe['25%']) & \
        (df_mtmv['med_hhinc'] <= mtmv_describe['50%'])] = '2 - Middle lower 25%'

df_mtmv['med_hhinc_quartile'].loc[(df_mtmv['med_hhinc'] >  mtmv_describe['50%']) & \
        (df_mtmv['med_hhinc'] <= mtmv_describe['75%'])] = '3 - Upper middle 25%'

df_mtmv['med_hhinc_quartile'].loc[df_mtmv['med_hhinc'] >  mtmv_describe['75%']] = '4 - Upper 25%'

Now let's check that we split up the data in a way that makes sense.

In [ ]:
df_mtmv['med_hhinc_quartile'].value_counts()

Now that we have two categorical variables, let's make the crosstab. A *crosstab* reports how many units (in this case, Congressional districts) fall into bins which share values. So, say we want to know how many districts had a Republican incumbent but are in the upper 25% income quartile. We can find this out for all income quartiles and incumbency conditions. 

In [ ]:
crosstab = pd.crosstab(df_mtmv['med_hhinc_quartile'], 
                       df_mtmv['rep_inc']) 

In [ ]:
crosstab

So it looks like of all of these, incomes in the lowest 25% quartile mostly did not have Republican incumbents.

It may be more interpretable to understand what percentage of Congressional districts each of these values represents. To do that, we can use the `normalize` keyword argument to normalize values across the whole crosstab.

In [ ]:
crosstab_all = pd.crosstab(df_mtmv['med_hhinc_quartile'], 
                           df_mtmv['rep_inc'], 
                           normalize = 'all') 

In [ ]:
crosstab_all

What if we want to compare the percentage of Republican incumbency across income classes? For instance, it may be the case I don't necessary care about overall percentage of districts, but I care about comparing whether more people in the lower 25% have a Republican incumbent or not. For this case, we can use `normalize = 'index'`.

In [ ]:
crosstab_index = pd.crosstab(df_mtmv['med_hhinc_quartile'], 
                             df_mtmv['rep_inc'], 
                             normalize = 'index')

In [ ]:
crosstab_index

Translating this into a heatmap is pretty simple. We pass the crosstab as the sole argument to the heatmap. It seems theoretically important to compare Republican incumbent districts to those which are not, so let's visualize `crosstab_index`.

**NB**: It is important to label your axises. If you don't label them, then you may have a hard time understanding what you are looking at when you revisie your code a few months later. We do this with `plt.xlabel` and `plt.ylabel`. They usually return something but we don't care about it. So we set it to a "throwaway" variable `_`.

In [ ]:
sns.heatmap(crosstab_index)
_ = plt.xlabel('Republican incumbency')
_ = plt.ylabel('Median Household Income Quartile')

This heatmap confirms what we saw either -- across household income levels, most districts did not have a Republican incumbent, and this is more pronounced in the Lower 25% of median household income.

Lastly, I find the colors in this really confusing. A color should probably be darker if the value is higher. Because of this, I'm going to pass another keyword argument, `cmap` to this function. I will give it the argument `Blues` which gives a series of blue-toned colors. The whole list of `matplotlib` colormaps can be found [at this page](https://matplotlib.org/examples/color/colormaps_reference.html).

In [ ]:
sns.heatmap(crosstab_index, cmap = 'Blues')
_ = plt.xlabel('Republican incumbency')
_ = plt.ylabel('Median Household Income Quartile')

### Swarmplots

Swarmplots are useful for when we want to visualize units split along some categorical and continuous variables. It also lets us visualize the sample size and if there's one or two results which might be messing up our average. In this case, we want to see if how Republican vote share is split between Republican and non-Republican incumbent districts. 

In [ ]:
sns.swarmplot('rep_inc', 'vote_share', data = df_mtmv)
_ = plt.xlabel('Republican incumbency')
_ = plt.ylabel('Republican vote share')

Unsurprisingly, Republican incumbent districts have a strong amount of Republican vote share.

Let's try this with Twitter mention share.

In [ ]:
sns.swarmplot('rep_inc', 'mshare', data = df_mtmv)
_ = plt.xlabel('Republican incumbency')
_ = plt.ylabel('Republican Twitter mention share')

It looks like districts which have Republican incumbents tend to tweet more about Republicans compared to those without a Republican incumbent.

### Violin plots and box plots

Violin plots and box plots are similar to swarm plots. While swarm plots illustrate every single data point, violin and box plots illustrate properties of the data distribution. Let's go back to the first example.

In [ ]:
a = df_mtmv[df_mtmv['rep_inc'] == 1]['vote_share'].values
a = a[~np.isnan(a)]

In [ ]:
sns.violinplot('rep_inc', 'vote_share', data = df_mtmv)
_ = plt.xlabel('Republican incumbency')
_ = plt.ylabel('Republican vote share')

In [ ]:
sns.boxplot('rep_inc', 'vote_share', data = df_mtmv)
_ = plt.xlabel('Republican incumbency')
_ = plt.ylabel('Republican vote share')

Let's take a second break down both of these plots. The violin plot displays a wideness to the data where there's more values. In this case it's like the histogram above. But it also displays a little more information. The white dot in the middle represents the median value of the variable. The thicker line in the middle are all values between 25th and 75th percentiles. The thinner line in the middle has a slightly [more technical definition](https://en.wikipedia.org/wiki/Box_plot) but it roughly corresponds to around the 10th and 90th percentile.

The boxplot (also called the box and whisker plot) is very similar. It shows the median. The box itself is the 25th and 75th percentile range. The whiskers correspond to the thin line in the violin plot. The dots beyond the whiskers are called "outliers" -- they are values which extend beyond the whiskers.

Violin and box plots have the advantage that they display more definitive information (like median, percentiles) about the data. They also don't display every data point, like swarmplots. This is helpful when it would be infeasible to display every point, e.g. when your data are quite large. Swarmplots will fail pretty badly for datasets above 300 rows or so.

### Scatter plots

A *scatter plot* shows how one continuous variable changes with another continuous variable. So for instance, we want to show how `mshare` changes with `vote_share`.

In [ ]:
_ = plt.plot(df_mtmv['mshare'], df_mtmv['vote_share'], marker = '.', linestyle = 'none')
_ = plt.xlabel('Republican mention share')
_ = plt.ylabel('Republican vote share')

We can begin to add in a third variable when we add in color. Color can indicate which observations take on a particular value in a variable. So say we want to also see which observations had Republican incumbents. 

We do this in a number of steps. First, let's create two new data frames, one with just Democrats and the other with just Republicans. Then, we will plot Republican incument districts and color them Red. We then plot only the rows which have non-Republican incumbents and color them Blue.

In [ ]:
df_rep = df_mtmv[df_mtmv['rep_inc'] == 1]
df_dem = df_mtmv[df_mtmv['rep_inc'] == 0]

_ = plt.plot(df_rep['mshare'], df_rep['vote_share'], color = 'Red', marker = '.', linestyle = 'none')
_ = plt.plot(df_dem['mshare'], df_dem['vote_share'], color = 'Blue', marker = '.', linestyle = 'none')

_ = plt.xlabel('Republican Twitter mention share')
_ = plt.ylabel('Republican vote share')

#### Exercise 1

1. Load the Adult Census data again. It's in `data/adult_census.csv`. Store the dataset in `df_adult`.
2. Describe the numerical details of `education-num`, `hours-per-week` and `age`.
3. Create a crosstab for `education` and `occupation`. 
4. Create a crosstab for `education` and `occupation` which compares percentages across occupations.
5. Create a heatmap for the above crosstab. Make sure to label the axises.
6. Create a violin plot of `income` versus `education-num`.
7. Create a scatter plot of `age` versus `education-num`.

## Modeling

A model is a simplified representation of reality. A mathematical model is a presentation of reality using numbers.

We use models for three purposes: 1) we want to summarize data, 2) we want to replicate the working of the world with existing data, and 3) we want to forecast the future from past data.

Modeling, in a sense, is a data reduction process. We are taking lots of existing data and reducing it to a few key metrics. Models are difficult to create because the real world is messy. To quote statistician George Box: "All models are wrong but some are useful."

In the past lab we described the data frame by understanding the structure of the data. We took simple measures of central tendency and dispersion. To begin to do some basic modelling, we can split up the data by a third variable.

To do this, we use a <code>groupby</code> which allows us to group by a variable or set of variables and apply some operation across them.

In [ ]:
## vote share and mention share mean 
## by Republican incumbency
gr_mtmv = df_mtmv.groupby('rep_inc')
gr_mtmv[['vote_share', 'mshare']].mean()

In [ ]:
## vote share and mention share standard deviation 
## by Republican incumbency
gr_mtmv[['vote_share', 'mshare']].std()

Now we can use a metric like [Pearson correlation](https://en.wikipedia.org/wiki/Pearson_product-moment_correlation_coefficient) to summarise the associated between two variables. Pearson correlation is measure of the linear dependence between two variables $X$ and $Y$. It ranges from [-1, 1]. It is typically represented by the Greek letter $\rho$ (pronounced rho).

In [ ]:
## this line removes rows with missing vote share data
df_mtmv = df_mtmv[pd.notnull(df_mtmv['vote_share'])]

## import the stats package
from scipy import stats

## calcuate
rho = stats.pearsonr(df_mtmv['mshare'], df_mtmv['vote_share'])
print(rho[0])

This returns two values in a tuple. The first is the Pearson correlation itself and we will print that out. The second is what's called a [p-value](https://en.wikipedia.org/wiki/P-value). The p-value is beyond the scope of this course so we will not discuss it too much more. The correlation is said to be *statistically significant* if its p-value is below a certain threshold, usually 0.05.

We can build a statistical model which attempts to summarise the data into a number of different metrics. This isn't a statistics course, so we won't get too much into statistics and the interpretation of statistical models. 

For this lab, we will be using *linear regression* to build a model for these data. Roughly, linear regression attempts to draw a trend line across one or more variables to fit the data. In the simplest example, we try to draw a line which summarizes the trend across two variables. This is called a *simple regression* or *bivariate regression*.

In [ ]:
a, b = np.polyfit(df_mtmv['mshare'], df_mtmv['vote_share'], deg = 1)
print("The slope of the line is ", a)
print("The intercept of the line is ", b)

Simple regression provides a slope and an intercept. The slope is a rough measure of association. If it's positive and closer to 1, then the two variables are connected. The intercept shifts the trend line up and down.

We can plot this against our scatter plot which we generated earlier.

In [ ]:
## produce the scatter plot
_ = plt.plot(df_mtmv['mshare'], df_mtmv['vote_share'], marker = '.', linestyle = 'none')

## set up two points to create a line
x = np.array([0, 100])
y = a * x + b

# Add regression line to your plot
_ = plt.plot(x, y)

## remember to label!
_ = plt.xlabel('Republican mention share')
_ = plt.ylabel('Republican vote share')

plt.show()

The more complicated version of this is the *multiple regression*. The multiple regression procedure creates a number of different slopes, in several different dimensions. Instead of calling them all slopes, we call them *coefficients*. Coefficients measure the effect of any particular variable. Like with correlation, the coefficient is said to be *statistically significant* if its p-value is below a certain threshold, usually 0.05.

Don't worry about most of the results presented below. However, look at the `coef` column and the `P>|t|` column to assess the coefficient and p-value.

In [ ]:
from statsmodels.formula.api import ols

model = ols("vote_share ~ mshare + rep_inc + mccain + pct_white + \
            pct_college + med_hhinc + pct_female", df_mtmv).fit()
model.summary()

As is expected, `rep_inc` has the largest coefficient with 11.06. Next is if the district voted for McCain in the 2008 election, followed by percent white in the population. All three have p-values far below 0.05. The next statistically significant effect is our variable of interest, Twitter mention share. It has a small effect which is statistcally significant at the p-value level of 0.01.

In [ ]:
df_adult

#### Exercise 2

1. Return to the Adult census dataset. Calculate the Pearson correlation between `education-num` and `capital-gain`.
2. Calculate the bivariate linear regression between `education-num` and `capital-gain`.
3. Complete this code to create the scatter plot and the regression line of these two variables.

In [ ]:
## produce the scatter plot
_ = plt.plot(____, ____, marker = '.', linestyle = 'none')

## set up two points to create a line
x = np.array([0, 20])
y = ____

# Add regression line to your plot
_ = plt.plot(x, y)

## remember to label!
_ = plt.xlabel(____)
_ = plt.ylabel(____)

plt.show()

<dd>4. It looks like this dataset is being skewed by some super rich people who are making 100,000 in capital gains. Complete the code below, which deletes the very rich. What changes?</dd> 

In [ ]:
a, b = ___

## produce the scatter plot
_ = plt.plot(___, ___, marker = '.', linestyle = 'none')

## set up two points to create a line
x = np.array([0, 20])
y = ___

# Add regression line to your plot
_ = plt.plot(x, y)

## remember to label!
_ = plt.xlabel(___)
_ = plt.ylabel(___)

plt.show()